## Carregando um arquivo csv

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell'

# from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession
spark = SparkSession.builder.master("local[1]").appName("gboaviagemApp").getOrCreate()

In [ ]:
sqlc = SQLContext(spark.sparkContext)

#### Arquivos:
```sh
(base) jovyan@7d6e9fae894c:~/work/Neo4jData$ ls
neo4j_edge_list-COMPROU.csv  neo4j_node_list-Produto.csv  neo4j_node_list-Usuario.csv
(base) jovyan@7d6e9fae894c:~/work/Neo4jData$ pwd
/home/jovyan/work/Neo4jData
```

In [20]:
wdir = "/home/jovyan/work/neo4jdata-200/"

n_users = spark.read.csv(wdir + "neo4j_node_list-Usuario.csv", header=True, inferSchema=True)
n_prods = spark.read.csv(wdir + "neo4j_node_list-Produto.csv", header=True, inferSchema=True)
e_comprou = spark.read.csv(wdir + "neo4j_edge_list-COMPROU.csv", header=True, inferSchema=True)\
    .select('email', 'produto_id', 'valor_pago', 'quando')

In [21]:
n_users.show()

+-----------------+---+
|            email| uf|
+-----------------+---+
|user010@gmail.com| CE|
|user007@gmail.com| AM|
|user008@gmail.com| MS|
|user003@gmail.com| MA|
|user001@gmail.com| PB|
|user004@gmail.com| MS|
|user002@gmail.com| PR|
|user006@gmail.com| AP|
|user005@gmail.com| PE|
|user009@gmail.com| SE|
+-----------------+---+



In [22]:
n_users.printSchema()

root
 |-- email: string (nullable = true)
 |-- uf: string (nullable = true)



In [23]:
n_prods.printSchema()

root
 |-- produto_id: string (nullable = true)



In [24]:
e_comprou.printSchema()

root
 |-- email: string (nullable = true)
 |-- produto_id: string (nullable = true)
 |-- valor_pago: double (nullable = true)
 |-- quando: string (nullable = true)



## Casting para Timestamp

In [25]:
from pyspark.sql import functions as F

e_comprou = e_comprou.withColumn('quando', F.current_timestamp())

In [26]:
e_comprou.printSchema()

root
 |-- email: string (nullable = true)
 |-- produto_id: string (nullable = true)
 |-- valor_pago: double (nullable = true)
 |-- quando: timestamp (nullable = false)



## Criando o GraphFrame

In [27]:
nodes1 = n_users.\
    drop_duplicates(subset=['email']).\
    withColumnRenamed('email', 'id').\
    withColumn('tipo', F.lit('Usuario'))

nodes2 = n_prods.\
    drop_duplicates().\
    withColumnRenamed('produto_id', 'id').\
    withColumn('tipo', F.lit('Produto'))

In [28]:
nodes1.count()

10

In [29]:
nodes2.count()

50

In [30]:
nodes = nodes1.join(nodes2, ['id', 'tipo'], 'outer')

In [31]:
nodes.printSchema()

root
 |-- id: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- uf: string (nullable = true)



In [32]:
nodes.count()

60

In [33]:
nodes.limit(5).show()

+---------+-------+----+
|       id|   tipo|  uf|
+---------+-------+----+
|Produto38|Produto|null|
| Produto9|Produto|null|
|Produto36|Produto|null|
| Produto8|Produto|null|
|Produto30|Produto|null|
+---------+-------+----+



In [34]:
e_comprou.count()

169

In [35]:
e_comprou.printSchema()

root
 |-- email: string (nullable = true)
 |-- produto_id: string (nullable = true)
 |-- valor_pago: double (nullable = true)
 |-- quando: timestamp (nullable = false)



In [36]:
edges = e_comprou.\
    withColumnRenamed('email', 'src').\
    withColumnRenamed('produto_id', 'dst').\
    withColumn('tipo', F.lit('COMPROU'))

In [37]:
edges.show()

+-----------------+---------+----------+--------------------+-------+
|              src|      dst|valor_pago|              quando|   tipo|
+-----------------+---------+----------+--------------------+-------+
|user010@gmail.com|Produto30|     148.0|2021-03-26 01:17:...|COMPROU|
|user007@gmail.com|Produto14|     165.0|2021-03-26 01:17:...|COMPROU|
|user010@gmail.com|Produto44|     153.0|2021-03-26 01:17:...|COMPROU|
|user008@gmail.com|Produto48|     196.0|2021-03-26 01:17:...|COMPROU|
|user003@gmail.com|Produto22|      89.0|2021-03-26 01:17:...|COMPROU|
|user001@gmail.com|Produto30|     148.0|2021-03-26 01:17:...|COMPROU|
|user004@gmail.com|Produto34|     108.0|2021-03-26 01:17:...|COMPROU|
|user001@gmail.com|Produto49|     182.0|2021-03-26 01:17:...|COMPROU|
|user002@gmail.com|Produto28|     172.0|2021-03-26 01:17:...|COMPROU|
|user007@gmail.com|Produto36|     130.0|2021-03-26 01:17:...|COMPROU|
|user006@gmail.com|Produto22|      89.0|2021-03-26 01:17:...|COMPROU|
|user005@gmail.com| 

In [38]:
from graphframes import GraphFrame

g = GraphFrame(nodes, edges)

In [39]:
q = g.find("(usuario)-[]->(prod)").withColumn('usuario', F.col('usuario').id)

In [40]:
# Top compradores
# q.groupBy('user').count().sort(F.col('count').desc(), 'user').show()
q = q.groupBy('usuario')\
    .agg(F.count('prod').alias('qtd_produtos'))\
    .sort(F.col('qtd_produtos').desc(), 'usuario')

In [41]:
q.show()

+-----------------+------------+
|          usuario|qtd_produtos|
+-----------------+------------+
|user003@gmail.com|          20|
|user005@gmail.com|          20|
|user007@gmail.com|          20|
|user002@gmail.com|          19|
|user004@gmail.com|          17|
|user006@gmail.com|          16|
|user008@gmail.com|          16|
|user009@gmail.com|          15|
|user010@gmail.com|          14|
|user001@gmail.com|          12|
+-----------------+------------+



In [ ]:
q.write.csv("top_compradores")

In [42]:
# Top produtos
q = g.find("(u)-[e]->(produto)") #.withColumn('user', F.col('user').id)

In [43]:
q = q.withColumn('valor_pago', F.col('e').valor_pago)\
    .groupBy("produto")\
    .agg(
        F.count("u").alias("qtd_compradores"),
        F.avg("valor_pago").alias("preco_medio"))\
    .withColumn('produto', F.col('produto').id)\
    .sort(F.col('qtd_compradores').desc(), 'produto')

In [44]:
q.show()

+---------+---------------+-----------+
|  produto|qtd_compradores|preco_medio|
+---------+---------------+-----------+
| Produto1|              6|      153.0|
|Produto16|              6|      191.0|
|Produto18|              6|      131.0|
|Produto36|              6|      130.0|
|Produto13|              5|      166.0|
| Produto2|              5|      140.0|
|Produto20|              5|      199.0|
|Produto24|              5|       87.0|
|Produto29|              5|      157.0|
| Produto3|              5|      182.0|
|Produto33|              5|      147.0|
|Produto34|              5|      108.0|
| Produto5|              5|      124.0|
|Produto50|              5|      130.0|
|Produto10|              4|      126.0|
|Produto12|              4|      130.0|
|Produto22|              4|       89.0|
|Produto28|              4|      172.0|
|Produto38|              4|       94.0|
| Produto4|              4|      171.0|
+---------+---------------+-----------+
only showing top 20 rows



### Recomendação em grão usuário (offline)

Dado um usuário A, quais os produtos comprados por usuários B que compraram os mesmos produtos que A?

In [54]:
q = g.find("(u1)-[]->(p1); (u2)-[]->(p1); (u2)-[]->(p2); !(u1)-[]->(p2)")\
    .withColumn('u1', F.col('u1').id)\
    .withColumn('u2', F.col('u2').id)\
    .withColumn('p1', F.col('p1').id)\
    .withColumn('p2', F.col('p2').id)\
    .where(
        (F.col('u1') == 'user003@gmail.com') &
        (F.col('u1') != F.col('u2')))

In [55]:
q.limit(5).show()

+-----------------+---------+-----------------+---------+
|               u1|       p1|               u2|       p2|
+-----------------+---------+-----------------+---------+
|user003@gmail.com|Produto22|user007@gmail.com|Produto36|
|user003@gmail.com| Produto1|user010@gmail.com|Produto30|
|user003@gmail.com| Produto2|user005@gmail.com| Produto6|
|user003@gmail.com|Produto11|user004@gmail.com|Produto17|
|user003@gmail.com|Produto24|user010@gmail.com|Produto18|
+-----------------+---------+-----------------+---------+



In [56]:
# Recomendações para user003 apenas a partir das compras de user005 (comparar com query Neo4j)
q.where(q.u2 == "user005@gmail.com")\
    .drop_duplicates(subset=['p2'])\
    .sort(F.col('p2'))\
    .show()

+-----------------+---------+-----------------+---------+
|               u1|       p1|               u2|       p2|
+-----------------+---------+-----------------+---------+
|user003@gmail.com|Produto20|user005@gmail.com|Produto15|
|user003@gmail.com|Produto20|user005@gmail.com|Produto23|
|user003@gmail.com|Produto13|user005@gmail.com|Produto28|
|user003@gmail.com|Produto12|user005@gmail.com| Produto3|
|user003@gmail.com|Produto34|user005@gmail.com|Produto36|
|user003@gmail.com|Produto20|user005@gmail.com| Produto4|
|user003@gmail.com| Produto1|user005@gmail.com|Produto43|
|user003@gmail.com| Produto1|user005@gmail.com|Produto48|
|user003@gmail.com| Produto2|user005@gmail.com|Produto49|
|user003@gmail.com| Produto2|user005@gmail.com| Produto6|
|user003@gmail.com|Produto16|user005@gmail.com| Produto8|
|user003@gmail.com|Produto24|user005@gmail.com| Produto9|
+-----------------+---------+-----------------+---------+



#### Recomendações numa lista Python

In [66]:
rec = q.select("p2").drop_duplicates()

In [69]:
rec_list = [row['p2'] for row in rec.collect()]
rec_list.sort()
print(rec_list

['Produto38',
 'Produto9',
 'Produto36',
 'Produto8',
 'Produto30',
 'Produto15',
 'Produto7',
 'Produto3',
 'Produto6',
 'Produto39',
 'Produto32',
 'Produto19',
 'Produto23',
 'Produto43',
 'Produto17',
 'Produto18',
 'Produto50',
 'Produto47',
 'Produto45',
 'Produto28',
 'Produto35',
 'Produto49',
 'Produto41',
 'Produto4',
 'Produto26',
 'Produto31',
 'Produto40',
 'Produto48',
 'Produto14',
 'Produto44']

In [62]:
rec_per_buyer = q.select("u2", "p2")\
    .groupBy("u2")\
    .agg(
        F.collect_set("p2"))\
    .withColumnRenamed("u2", "compradores")\
    .withColumnRenamed("collect_set(p2)", "tambem_compraram")

In [63]:
rec.toPandas()

,compradores,tambem_compraram
0,user005@gmail.com,"[Produto3, Produto36, Produto8, Produto49, Pro..."
1,user004@gmail.com,"[Produto18, Produto17, Produto3, Produto8, Pro..."
2,user007@gmail.com,"[Produto18, Produto17, Produto50, Produto36, P..."
3,user002@gmail.com,"[Produto18, Produto3, Produto47, Produto19, Pr..."
4,user006@gmail.com,"[Produto45, Produto18, Produto3, Produto50, Pr..."
5,user010@gmail.com,"[Produto18, Produto50, Produto41, Produto36, P..."
6,user008@gmail.com,"[Produto47, Produto50, Produto8, Produto49, Pr..."
7,user009@gmail.com,"[Produto18, Produto50, Produto36, Produto47, P..."
8,user001@gmail.com,"[Produto3, Produto47, Produto8, Produto49, Pro..."


In [57]:
q.write.csv('graph_recomendacao_usuario003.csv')